In [4]:
#Import libraries
import pandas as pd
!pip install jupyter-dash
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import plotly.express as px
from dash import no_update

In [7]:
#Read the airline data
airline_data =  pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/Data%20Files/airline_data.csv', 
                            encoding = "ISO-8859-1",
                            dtype={'Div1Airport': str, 'Div1TailNum': str, 
                                   'Div2Airport': str, 'Div2TailNum': str})
airline_data

,Unnamed: 0,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,...,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1295781,1998,2,4,2,4,1998-04-02,AS,19930,AS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1125375,2013,2,5,13,1,2013-05-13,EV,20366,EV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,118824,1993,3,9,25,6,1993-09-25,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,634825,1994,4,11,12,6,1994-11-12,HP,19991,HP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1888125,2017,3,8,17,4,2017-08-17,UA,19977,UA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26995,821542,2017,1,1,24,2,2017-01-24,DL,19790,DL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26996,1910565,2013,2,6,27,4,2013-06-27,B6,20409,B6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26997,9055,2016,3,8,26,5,2016-08-26,AA,19805,AA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26998,84136,2009,3,8,8,6,2009-08-08,YV,20378,YV,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
airline_data.info

<bound method DataFrame.info of        Unnamed: 0  Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate  \
0         1295781  1998        2      4           2          4  1998-04-02   
1         1125375  2013        2      5          13          1  2013-05-13   
2          118824  1993        3      9          25          6  1993-09-25   
3          634825  1994        4     11          12          6  1994-11-12   
4         1888125  2017        3      8          17          4  2017-08-17   
...           ...   ...      ...    ...         ...        ...         ...   
26995      821542  2017        1      1          24          2  2017-01-24   
26996     1910565  2013        2      6          27          4  2013-06-27   
26997        9055  2016        3      8          26          5  2016-08-26   
26998       84136  2009        3      8           8          6  2009-08-08   
26999      113029  1993        3      7          17          6  1993-07-17   

      Reporting_Airline  DOT_ID

In [ ]:
#List of years
year_list = [i for i in range(2005, 2021, 1)]

def compute_data_choice_1(df):
    #Cancellation Category Count for Bar Chart
    bar_data = df.groupby(['Month','CancellationCode'])['Flights'].sum().reset_index()
    #Average flight time by reporting airline
    line_data = df.groupby(['Month','Reporting_Airline'])['AirTime'].mean().reset_index()
    #Diverted Airport Landings
    div_data = df[df['DivAirportLandings'] != 0.0]
    #Source state count
    map_data = df.groupby(['OriginState'])['Flights'].sum().reset_index()
    #Destination state count
    tree_data = df.groupby(['DestState', 'Reporting_Airline'])['Flights'].sum().reset_index()
    return bar_data, line_data, div_data, map_data, tree_data

def compute_data_choice_2(df):
    #Compute delay average 
    avg_car = df.groupby(['Month','Reporting_Airline'])['CarrierDelay'].mean().reset_index()
    avg_weather = df.groupby(['Month','Reporting_Airline'])['WeatherDelay'].mean().reset_index()
    avg_NAS = df.groupby(['Month','Reporting_Airline'])['NASDelay'].mean().reset_index()
    avg_sec = df.groupby(['Month','Reporting_Airline'])['SecurityDelay'].mean().reset_index()
    avg_late = df.groupby(['Month','Reporting_Airline'])['LateAircraftDelay'].mean().reset_index()
    return avg_car, avg_weather, avg_NAS, avg_sec, avg_late 

##Create dash application and clear the layout until callback executed
app = dash.Dash(__name__)

##Set the application to show the graphs after callback
app.config.suppress_callback_exceptions = True


# Application layout
app.layout = html.Div(children=[
                                html.H1('US Domestic Airline Flights Performance', 
                                style={'textAlign': 'center', 'color': '#503D36',
                                'font-size': 24}),
                                
                                #Outer division
                                html.Div([
                                    #Inner division 1
                                    html.Div([
                                        html.Div(
                                            [html.H2('Report Type:', style={'margin-right': '2em'}),  ]
                                                ),
                                         dcc.Dropdown(id='input-type', 
                                                    options=[
                                                    {'label': 'Yearly Airline Performance Report', 'value': 'OPT1'},
                                                    {'label': 'Yearly Airline Delay Report', 'value': 'OPT2'}
                                                    ],
                                                    placeholder='Select a report type',
                                                    style={'width': '80%', 'padding': '3px', 'font size': '20px', 'text-align-last': 'center'})], 
                                                    style={'display':'flex'}),
                                    
                                    #Inner division 2
                                    html.Div([
                                         html.Div(
                                            [
                                            html.H2('Choose Year:', style={'margin-right': '2em'}),
                                            ]
                                                ),
                                        dcc.Dropdown(id='input-year', 
                                                     # Update dropdown values using list comphrehension
                                                     options=[{'label': i, 'value': i} for i in year_list],
                                                     placeholder="Select a year",
                                                     style={'width':'80%', 'padding':'3px', 'font-size': '20px', 'text-align-last' : 'center'}),
                                            # Place them next to each other using the division style
                                            ], style={'display': 'flex'}),  
                                          ]),
    # Add computer graphs to be updated during callback
                                    #1st division
                                    html.Div([ ], id='plot1'),
                                    #2nd division
                                    html.Div([
                                            html.Div([ ], id='plot2'),
                                            html.Div([ ], id='plot3')
                                    ], style={'display': 'flex'}),
                                    #3rd division
                                    html.Div([
                                            html.Div([ ], id='plot4'),
                                            html.Div([ ], id='plot5')
                                    ], style={'display': 'flex'})])

@app.callback([Output(component_id='plot1', component_property='children'),
               Output(component_id='plot2', component_property='children'),
               Output(component_id='plot3', component_property='children'),
               Output(component_id='plot4', component_property='children'),
               Output(component_id='plot5', component_property='children')],
              [Input(component_id='input-type', component_property='value'),
               Input(component_id='input-year', component_property='value')],
               #Holding output state till user enters all the form information (type of chart and year)
              [State("plot1", "children"), State("plot2", "children"),
               State("plot3", "children"), State("plot4", "children"),
               State("plot5", "children")
              ])
def get_graph(chart, year, children1, children2, c3, c4, c5):
      
        # Select data
        df =  airline_data[airline_data['Year']== year]
       
        if chart == 'OPT1':
            # Compute required information for creating graph from the data
            bar_data, line_data, div_data, map_data, tree_data = compute_data_choice_1(df)
            
            # Number of flights under different cancellation categories
            bar_fig = px.bar(bar_data, x='Month', y='Flights', color='CancellationCode', title='Monthly Flight Cancellation')
            
            # Average flight time by reporting airline
            line_fig = px.line(line_data, x ='Month', y='AirTime', color = 'Reporting_Airline', title = 'Average monthly flight time (minutes) by airline.')
            
            # Percentage of diverted airport landings per reporting airline
            pie_fig = px.pie(div_data, values='Flights', names='Reporting_Airline', title='% of flights by reporting airline')
            
            # Number of flights flying from each state using choropleth
            map_fig = px.choropleth(map_data,  # Input data
                    locations='OriginState', 
                    color='Flights',  
                    hover_data=['OriginState', 'Flights'], 
                    locationmode = 'USA-states', # Set to plot as US States
                    color_continuous_scale='GnBu',
                    range_color=[0, map_data['Flights'].max()]) 
            map_fig.update_layout(
                    title_text = 'Number of flights from origin state', 
                    geo_scope='usa') # Plot only the USA instead of globe
            
            # Number of flights flying to each state from each reporting airline
            tree_fig = px.treemap(tree_data, path=['DestState', 'Reporting_Airline'], 
                      values='Flights',
                      color='Reporting_Airline',
                      color_continuous_scale='RdBu',
                      title='Flight count by airline to destination state'
                )
            return [dcc.Graph(figure=tree_fig), 
                    dcc.Graph(figure=pie_fig),
                    dcc.Graph(figure=map_fig),
                    dcc.Graph(figure=bar_fig),
                    dcc.Graph(figure=line_fig)
                   ]
            
        else:
            # Charts under "Flight Delay Time Statistics Dashboard" section
            # Compute required information for creating graph from the data
            avg_car, avg_weather, avg_NAS, avg_sec, avg_late = compute_data_choice_2(df)
            
            # Create graph
            carrier_fig = px.line(avg_car, x='Month', y='CarrierDelay', color='Reporting_Airline', title='Average carrrier delay time (minutes) by airline')
            weather_fig = px.line(avg_weather, x='Month', y='WeatherDelay', color='Reporting_Airline', title='Average weather delay time (minutes) by airline')
            nas_fig = px.line(avg_NAS, x='Month', y='NASDelay', color='Reporting_Airline', title='Average NAS delay time (minutes) by airline')
            sec_fig = px.line(avg_sec, x='Month', y='SecurityDelay', color='Reporting_Airline', title='Average security delay time (minutes) by airline')
            late_fig = px.line(avg_late, x='Month', y='LateAircraftDelay', color='Reporting_Airline', title='Average late aircraft delay time (minutes) by airline')
            
            return[dcc.Graph(figure=carrier_fig), 
                   dcc.Graph(figure=weather_fig), 
                   dcc.Graph(figure=nas_fig), 
                   dcc.Graph(figure=sec_fig), 
                   dcc.Graph(figure=late_fig)]
if __name__ == '__main__':
    app.run_server(debug=True)
                                        

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


OSError: [Errno 48] Address already in use